<div style="background-color: beige; padding: 10px;">
<p style='font-size: 22px; text-align: right;'><strong>Stephanie Dragoi and Victor Matsumoto</strong></p>
<p style='font-size: 22px; text-align: right;'><strong>DUE 20231026</strong></p>
<p style='font-size: 22px; text-align: right;'><strong>MLR Project- Used Car Pricing Model</strong></p>
</div>

# Introduction and Car Selection

[include information about munging]

# Step 4: Import Libraries

In [1]:
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip

# Step 5: Python Functions

In [2]:
# Define useful functions including regression_performance, calculate_vif

def regression_performance(model, X, y):
 yhat = model.predict(X)
 mse = mean_squared_error(y, yhat)
 mae = mean_absolute_error(y, yhat)
 mape = np.mean(np.abs(y - yhat) / y) * 100 # mean absolute pct. error
 df_performance_summary = pd.DataFrame(
 {"MSE": mse, "MAE": mae, "MAPE": mape}, index=[0])
 return df_performance_summary

# Function to return a dataframe showing the VIF for each explanatory variable:
# Note that the explanatory variables are in a dataframe called Xdataframe.
def calculate_vif(Xdataframe):
 temp_df = pd.DataFrame()
 temp_df["column name"] = Xdataframe.columns
 temp_df["VIF"] = [variance_inflation_factor(Xdataframe.values, k)
 for k in range(len(Xdataframe.columns))]
 return temp_df

# Step 6: Dataframe Summary Statistics

In [4]:
# read csv and describe
df = pd.read_csv('Munged Car Data.csv')
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,288.0,2010.899306,2.931316,2003.000000,2009.00000,2011.00000,2013.00000,2018.0000
asking_price,288.0,8961.966146,4168.622707,2000.000000,5699.96250,8500.00000,11424.95000,22500.0000
miles_driven,288.0,60080.651093,27552.615863,7454.545455,43484.84848,55909.09091,74545.45455,168348.4848
num_previous_owners,288.0,1.663194,0.733304,1.000000,1.00000,2.00000,2.00000,4.0000
seats,288.0,5.000000,0.000000,5.000000,5.00000,5.00000,5.00000,5.0000


# Step 7: Variable Types and Missing Data

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make                 288 non-null    object 
 1   model                288 non-null    object 
 2   model_contd          288 non-null    object 
 3   year                 288 non-null    int64  
 4   asking_price         288 non-null    float64
 5   miles_driven         288 non-null    float64
 6   fuel                 288 non-null    object 
 7   seller_type          288 non-null    object 
 8   transmission         288 non-null    object 
 9   num_previous_owners  288 non-null    int64  
 10  mileage              288 non-null    object 
 11  engine               288 non-null    object 
 12  max_power            288 non-null    object 
 13  torque               288 non-null    object 
 14  seats                288 non-null    int64  
dtypes: float64(2), int64(3), object(10)
memo